In [11]:
import os
import glob
import numpy as np
import pandas as pd
import sys
import re
import ast
import concurrent.futures


In [12]:
# --- 사용자 설정 (하이퍼파라미터) ---
DATA_PATH = '/dataset/usr012/jmpark/Moudlar_data/data'
TRAIN_OUTPUT_PATH = '/dataset/usr012/jmpark/Modular_train'
EXPERT_OUTPUT_PATH = '/dataset/usr012/jmpark/Modular_train/expert'

# 500만 스텝 중 저장할 스텝 수
MAX_STEPS_TO_SAVE = 3_000_000
MAX_WORKERS = max(1, os.cpu_count() - 10)
# ---------------------------------

In [13]:
def process_csv_to_npz(file_path, output_dir, max_steps, file_type):
    """
    단일 .csv.gz 파일을 읽어 지정된 포맷의 .npz 파일로 변환합니다.
    file_type에 따라 다른 이름 정규화 규칙을 적용합니다.
    """
    try:
        filename = os.path.basename(file_path)
        if file_type == 'train':
            # Train 파일: _날짜_시간_train 을 통째로 제거
            pattern = r'_\d{8}_\d{6}_train'
            cleaned_filename = re.sub(pattern, '', filename)
        elif file_type == 'expert':
            # Expert 파일: _날짜_시간 만 제거 (expert는 남김)
            pattern = r'_\d{8}_\d{6}'
            cleaned_filename = re.sub(pattern, '', filename)
        else:
            # 예외 처리
            cleaned_filename = filename
        npz_filename = cleaned_filename.replace('.csv.gz', '.npz')
        
        output_filepath = os.path.join(output_dir, npz_filename)
        
        print(f"[{filename}] 처리 시작...")
        
        # 1. CSV 로드
        converters = {
            'state': ast.literal_eval,
            'action': ast.literal_eval,
            'next_state': ast.literal_eval
        }
        use_cols = ['state', 'action', 'next_state', 'reward', 'done_code']
        
        try:
            df = pd.read_csv(
                file_path, 
                compression='gzip',
                nrows=max_steps,
                usecols=use_cols,
                converters=converters
            )
        except Exception as e:
            print(f"  [오류] {filename} 파일 읽기 실패: {e}", file=sys.stderr)
            return f"{filename} (실패)"

        # 2. 차원 정보 추출
        try:
            state_dim = len(df['state'].iloc[0])
            action_dim = len(df['action'].iloc[0])
            next_state_dim = len(df['next_state'].iloc[0])
            
            if state_dim != next_state_dim:
                print(f"  [오류] {filename}의 state({state_dim})와 "
                      f"next_state({next_state_dim}) 차원이 다릅니다.", file=sys.stderr)
                return f"{filename} (실패)"
                
        except (IndexError, TypeError) as e:
            print(f"  [오류] {filename}의 state/action 차원 파악 실패: {e}", file=sys.stderr)
            return f"{filename} (실패)"
            
        # 3. transitions 배열 생성
        s_arr = np.array(df['state'].to_list(), dtype=np.float32)
        a_arr = np.array(df['action'].to_list(), dtype=np.float32)
        ns_arr = np.array(df['next_state'].to_list(), dtype=np.float32)
        r_arr = df['reward'].to_numpy(dtype=np.float32).reshape(-1, 1)
        d_arr = df['done_code'].to_numpy(dtype=np.float32).reshape(-1, 1)
        transitions = np.hstack((s_arr, a_arr, ns_arr, r_arr, d_arr))

        # 4. NPZ 파일로 압축 저장
        np.savez_compressed(
            output_filepath,
            transitions=transitions,
            state_dim=state_dim,
            action_dim=action_dim
        )
        
        print(f"  -> {output_filepath} 저장 완료. (shape={transitions.shape}, s_dim={state_dim}, a_dim={action_dim})")
        return f"{filename} (성공)"

    except Exception as e:
        print(f"  [알 수 없는 오류] {filename} 처리 중 실패: {e}", file=sys.stderr)
        return f"{filename} (실패)"

In [ ]:
def main():
    # 1. 출력 디렉토리 생성 (존재하면 무시)
    os.makedirs(TRAIN_OUTPUT_PATH, exist_ok=True)
    os.makedirs(EXPERT_OUTPUT_PATH, exist_ok=True)

    # 2. train 파일 변환
    train_files = glob.glob(os.path.join(DATA_PATH, '*train.csv.gz'))
    if not train_files:
        print(f"경고: '{DATA_PATH}'에서 *train.csv.gz 파일을 찾을 수 없습니다.")
    else:
        print(f"\n--- 총 {len(train_files)}개의 Train 파일 변환 시작 ---")
        for file in sorted(train_files):
            process_csv_to_npz(file, TRAIN_OUTPUT_PATH, MAX_STEPS_TO_SAVE)

    # 3. expert 파일 변환
    expert_files = glob.glob(os.path.join(DATA_PATH, '*expert.csv.gz'))
    if not expert_files:
        print(f"경고: '{DATA_PATH}'에서 *expert.csv.gz 파일을 찾을 수 없습니다.")
    else:
        print(f"\n--- 총 {len(expert_files)}개의 Expert 파일 변환 시작 ---")
        for file in sorted(expert_files):
            process_csv_to_npz(file, EXPERT_OUTPUT_PATH, MAX_STEPS_TO_SAVE)

    print("\n--- 모든 작업 완료 ---")

if __name__ == "__main__":
    main()

In [14]:
def main():
    os.makedirs(TRAIN_OUTPUT_PATH, exist_ok=True)
    os.makedirs(EXPERT_OUTPUT_PATH, exist_ok=True)

    # 1. 처리할 파일 목록을 미리 준비합니다.
    train_files = glob.glob(os.path.join(DATA_PATH, '*train.csv.gz'))
    expert_files = glob.glob(os.path.join(DATA_PATH, '*expert.csv.gz'))
    
    total_files = len(train_files) + len(expert_files)
    if total_files == 0:
        print(f"경고: '{DATA_PATH}'에서 변환할 *.csv.gz 파일을 찾을 수 없습니다.")
        return
        
    print(f"총 {total_files}개의 파일 변환을 시작합니다. (최대 {MAX_WORKERS}개 코어 사용)")

    # 2. ProcessPoolExecutor를 생성합니다.
    with concurrent.futures.ProcessPoolExecutor(max_workers=MAX_WORKERS) as executor:
        
        futures = [] # 작업 목록을 저장할 리스트
        
        print(f"\n--- {len(train_files)}개의 Train 파일 작업 제출 ---")
        for file in sorted(train_files):
            futures.append(
                executor.submit(
                    process_csv_to_npz, 
                    file, 
                    TRAIN_OUTPUT_PATH, 
                    MAX_STEPS_TO_SAVE, 
                    'train'  # <--- 'train' 타입 지정
                )
            )

        print(f"\n--- {len(expert_files)}개의 Expert 파일 작업 제출 ---")
        for file in sorted(expert_files):
            futures.append(
                executor.submit(
                    process_csv_to_npz, 
                    file, 
                    EXPERT_OUTPUT_PATH, 
                    MAX_STEPS_TO_SAVE, 
                    'expert' # <--- 'expert' 타입 지정
                )
            )
        # 5. 작업이 완료될 때마다 결과 확인
        print("\n--- 작업 처리 시작 (로그가 섞여서 나올 수 있습니다) ---")
        success_count = 0
        fail_count = 0
        
        for future in concurrent.futures.as_completed(futures):
            result = future.result() 
            if "(성공)" in result:
                success_count += 1
            else:
                fail_count += 1
        
        print(f"\n--- 모든 작업 완료 (총 {total_files}개 중 {success_count}개 성공, {fail_count}개 실패) ---")

if __name__ == "__main__":
    main()

총 76개의 파일 변환을 시작합니다. (최대 38개 코어 사용)

--- 38개의 Train 파일 작업 제출 ---



--- 38개의 Expert 파일 작업 제출 ---

--- 작업 처리 시작 (로그가 섞여서 나올 수 있습니다) ---
[cheetah_4_allback_20251018_164153_train.csv.gz] 처리 시작...[cheetah_2_back_20251018_163515_train.csv.gz] 처리 시작...[cheetah_3_front_20251018_163907_train.csv.gz] 처리 시작...[cheetah_4_allfront_20251018_164234_train.csv.gz] 처리 시작...[cheetah_3_back_20251018_163702_train.csv.gz] 처리 시작...[cheetah_2_front_20251018_163607_train.csv.gz] 처리 시작...[cheetah_3_balanced_20251018_163753_train.csv.gz] 처리 시작...[cheetah_5_back_20251018_164602_train.csv.gz] 처리 시작...[cheetah_5_balanced_20251018_164649_train.csv.gz] 처리 시작...[cheetah_4_back_20251018_164319_train.csv.gz] 처리 시작...
[cheetah_5_front_20251018_164738_train.csv.gz] 처리 시작...




[cheetah_4_front_20251018_164427_train.csv.gz] 처리 시작...
[cheetah_6_back_20251018_164832_train.csv.gz] 처리 시작...


[hopper_3_20251018_165054_train.csv.gz] 처리 시작...[cheetah_6_front_20251018_164912_train.csv.gz] 처리 시작...[hopper_4_20251018_165126_train.csv.gz] 처리 시작...
[cheetah_7_full_20251018_165009_train.csv.gz] 처리 

In [18]:
import numpy as np
import os

# --- ★★★ 설정 ★★★ ---
# 방금 생성된 파일 중 하나를 골라 경로를 정확히 입력하세요.
# (예시: 'cheetah_2_back.npz'가 생성되었다고 가정)
TRAIN_OUTPUT_PATH = '/dataset/usr012/jmpark/Modular_train'
FILE_TO_TEST = os.path.join(TRAIN_OUTPUT_PATH, 'humanoid_2d_9_full.npz') 
# ------------------------


def test_npz_file(file_path):
    print(f"--- NPZ 파일 테스트 시작 ---")
    print(f"파일 경로: {file_path}\n")

    try:
        # 1. 파일 로드
        # np.load()는 'lazy loading'을 지원하므로 with 구문을 사용하는 것이 좋습니다.
        with np.load(file_path) as data:
            
            # 2. 저장된 모든 키 확인
            keys = list(data.keys())
            print(f"[키 목록 확인]")
            print(f"  -> {keys}")
            
            if 'transitions' not in keys or 'state_dim' not in keys or 'action_dim' not in keys:
                print("\n[오류] 'transitions', 'state_dim', 'action_dim' 중 \
                      하나 이상의 키가 없습니다.")
                return

            print("\n--- 각 키 상세 정보 ---")

            # 3. state_dim, action_dim 확인
            # .item()을 사용해야 numpy array(0) -> python int 0 으로 꺼내집니다.
            state_dim = data['state_dim'].item()
            action_dim = data['action_dim'].item()
            
            print(f"[state_dim]")
            print(f"  -> 값: {state_dim} (Type: {type(state_dim)})")
            
            print(f"[action_dim]")
            print(f"  -> 값: {action_dim} (Type: {type(action_dim)})")

            # 4. transitions 데이터 확인
            transitions = data['transitions']
            
            print(f"\n[transitions]")
            print(f"  -> 데이터 타입: {transitions.dtype}")
            print(f"  -> 전체 Shape: {transitions.shape}")
            print(f"  -> 첫 샘플 Shape: {transitions[0].shape}")

            # 5. Shape 유효성 검사
            # (행 개수, 열 개수)
            num_rows, num_cols = transitions.shape
            expected_cols = state_dim + action_dim + state_dim + 1 + 1 # (s, a, s', r, d)
            
            print(f"\n--- 유효성 검사 ---")
            print(f"  -> 행(스텝 수): {num_rows} (최대 {MAX_STEPS_TO_SAVE}개)")
            print(f"  -> 열(데이터 차원): {num_cols}")
            print(f"  -> 기대한 열: {expected_cols} (s:{state_dim} + a:{action_dim} + s':{state_dim} + r:1 + d:1)")

            if num_cols == expected_cols:
                print(f"  -> [성공] 열(Column) 개수가 일치합니다.")
            else:
                print(f"  -> [실패] 열(Column) 개수가 일치하지 않습니다!")

            print(f"\n--- 샘플 데이터 (첫 3줄) ---")
            print(transitions[:3])
            
            print("\n--- 테스트 완료 ---")

    except FileNotFoundError:
        print(f"[오류] 파일을 찾을 수 없습니다: {file_path}")
    except Exception as e:
        print(f"[오류] 파일 로드 중 에러 발생: {e}")

if __name__ == "__main__":
    # 이전 스크립트에서 설정한 MAX_STEPS_TO_SAVE 변수 (없으면 하드코딩)
    try:
        MAX_STEPS_TO_SAVE = MAX_STEPS_TO_SAVE
    except NameError:
        MAX_STEPS_TO_SAVE = 3_000_000 # 변환 스크립트의 설정값과 동일하게
        
    test_npz_file(FILE_TO_TEST)

--- NPZ 파일 테스트 시작 ---
파일 경로: /dataset/usr012/jmpark/Modular_train/humanoid_2d_9_full.npz

[키 목록 확인]
  -> ['transitions', 'state_dim', 'action_dim']

--- 각 키 상세 정보 ---
[state_dim]
  -> 값: 171 (Type: <class 'int'>)
[action_dim]
  -> 값: 8 (Type: <class 'int'>)

[transitions]
  -> 데이터 타입: float32
  -> 전체 Shape: (3000000, 352)
  -> 첫 샘플 Shape: (352,)

--- 유효성 검사 ---
  -> 행(스텝 수): 3000000 (최대 3000000개)
  -> 열(데이터 차원): 352
  -> 기대한 열: 352 (s:171 + a:8 + s':171 + r:1 + d:1)
  -> [성공] 열(Column) 개수가 일치합니다.

--- 샘플 데이터 (첫 3줄) ---
[[0.         0.         1.4020338  ... 0.63888896 1.0603821  0.        ]
 [0.         0.         1.4017417  ... 0.63888896 1.777074   0.        ]
 [0.         0.         1.4007542  ... 0.63888896 1.03553    0.        ]]

--- 테스트 완료 ---
